In [1]:
#BANK TELLER SCHEDULING OPTIMIZER
#TEAM MEMBERS:
    # Srivatsava Chinnasamy Kamaraj,
    # Manibalan Amaranathan
    

## Project Overview:
## This project is inspired by a classical **linear programming problem** found in Heizer, J., Render, B. and Munson, C., 2020. Operations Management: Sustainability and Supply Chain Management. 12th ed. Harlow: Pearson Education, Chapter 8  
## The Hong Kong Bank of Commerce and Industry case study, the labor scheduling example (Table 8.4)
## We have implemented a **brute-force approach**, but enhanced it with a **custom pruning strategy** developed by our team to dramatically reduce the number of loops and invalid evaluations.
## While we acknowledge that more efficient or mathematically optimal methods exist, our goal was to:
  ## Understand the logic from the ground up
  ## Solve the problem using core Python structures thought in the class (loops, conditionals, dictionaries)
  ## Develop practical and intelligent constraints to prune the search space efficiently


## PROBLEM:
    ##Hong Kong Bank of Commerce and Industry requires between 10 and 18 tellers depending on the time of day
    ##The bank wants a schedule that will minimize total personnel costs
    ##Lunch time from noon to 2 pm is generally the busiest
    ##Bank employs 12 full-time tellers, many part-time workers
    ##Part-time workers must put in exactly four hours per day, can start anytime between 9 am and 1 pm, and are inexpensive
    ##Full-time workers work from 9 am to 5 pm and have 1 hour for lunch
    ##Part-time hours are limited to a maximum of 50% of the day’s total requirements
    ##Part-timers earn $8 per hour on average
    ##Full-timers earn $100 per day on average
    ##It will release one or more of its full-time tellers if it is profitable to do so

##Example given in the question:
#Enter hourly teller demand from 9 AM to 5 PM (8 hours total):
#  Hour 9:00 - 10:00 → 10
#  Hour 10:00 - 11:00 → 12
#  Hour 11:00 - 12:00 → 14
#  Hour 12:00 - 13:00 → 16
#  Hour 13:00 - 14:00 → 18
#  Hour 14:00 - 15:00 → 17
#  Hour 15:00 - 16:00 → 15
#  Hour 16:00 - 17:00 → 10


# Step 1: Define hourly demand
hourly_demand = {}
print("Enter hourly teller demand from 9 AM to 5 PM (8 hours total):")
print("Note: demand range b/w 0 to 19")
for h in range(9, 17): # Loop over 8 hourly slots: 9am to 5pm
    while True:
        try:
            demand = int(input(f"  Hour {h}:00 - {h+1}:00 → ")) #input demand for each hour
            if demand < 0 or demand > 18:# Validate the demand is between 0 and 18
                raise ValueError
            hourly_demand[h] = demand
            break
        except ValueError:
            print("    Please enter a valid demand.")

# Step 2: Define part-time shift blocks
# Each part-timer works 4 continuous hours starting between 9am and 1pm
# Example: part-timer starting at 9 will work [9,10,11,12]
parttime_slots = {time: [time + i for i in range(4)] for time in range(9, 14)}

# Step 3: Contribution functions
# Calculates the hourly availability of full-time tellers (F) from 9 AM to 5 PM
# Full-timers work 9–5 (8 hours), but only half are available during 12–1 & 1-2 lunch break
# Full-timers take lunch in staggered shifts:
#   - Half go to lunch from 12 PM to 1 PM (hour 12)
#   - The other half go from 1 PM to 2 PM (hour 13)
# If the number of full-timers is odd, the extra person is assigned to hour 13
# This gives slightly more coverage at 1–2 PM than 12–1 PM when F is odd
def get_fulltime_contrib(F):
    return { h: (F // 2 if h == 12 else F // 2 + (F % 2) if h == 13 else F) for h in range(9, 17)}
    
# Part-time contribution per hour from the number of part-timers starting each shift
def get_parttime_contrib(P):
    part_time_contrib = {hour: 0 for hour in range(9, 17)}
    for start in P:
        for hour in parttime_slots[start]:
            part_time_contrib[hour] += P[start]
    return part_time_contrib
    
# Combine full-time and part-time coverage
def combine_contrib(ft, pt):
    return {h: ft[h] + pt[h] for h in hourly_demand}
    
# Check if all demand is covered in each hour
def is_valid_schedule(total_coverage):
    return all(total_coverage[h] >= hourly_demand[h] for h in hourly_demand)

# Step 4: Constraints
total_demand_hours = sum(hourly_demand.values()) # Total demand in all hours
max_pt_hours = total_demand_hours // 2 # Part-timers can only cover up to 50% of the total demand
max_part_timers = max_pt_hours // 4  # # Since each part-timer works 4 hours, max allowed part-timers
max_shift_pt = max_part_timers  # # Max part-timers that can be assigned to any one shift

# Step 5: Generate valid combinations with stepwise pruning for reducing unwanted loops in the bruteforce approach for-- optimizer(step 6)
# Brute-force loop over all possible combinations of 0 to max_shift_pt for each shift
# Stops early if sum of assigned part-timers exceeds max_part_timers
valid_combinations = []
for p9 in range(max_shift_pt + 1):
    if p9 > max_part_timers:
        continue
    for p10 in range(max_shift_pt + 1):
        if p9 + p10 > max_part_timers:
            continue
        for p11 in range(max_shift_pt + 1):
            if p9 + p10 + p11 > max_part_timers:
                continue
            for p12 in range(max_shift_pt + 1):
                if p9 + p10 + p11 + p12 > max_part_timers:
                    continue
                for p13 in range(max_shift_pt + 1):
                    if p9 + p10 + p11 + p12 + p13 > max_part_timers:
                        continue
                    valid_combinations.append((p9, p10, p11, p12, p13))  # If within allowed range, save the combination

# Step 6: Optimizer
best_cost = float('inf')
best_combo = None
best_total_coverage = None

for F in range(0, 13):  # # Try every possible number of full-time tellers (0 to 12)
    for combo in valid_combinations:
        p9, p10, p11, p12, p13 = combo
        P = {9: p9, 10: p10, 11: p11, 12: p12, 13: p13}
        total_pt = sum(combo)
        total_pt_hours = total_pt * 4

        if total_pt_hours > max_pt_hours: # Skip if this combo exceeds allowed part-time hours
            continue

        ft_contrib = get_fulltime_contrib(F)
        pt_contrib = get_parttime_contrib(P)
        total_coverage = combine_contrib(ft_contrib, pt_contrib)

        if is_valid_schedule(total_coverage): # Check if schedule is valid and better than current best
            cost = 100 * F + 32 * total_pt
            if cost < best_cost:
                best_cost = cost
                best_combo = {"F": F, **P}
                best_total_coverage = total_coverage
# Print the output
if best_combo is None:# If no valid combination found, notify
    print("No valid staffing schedule found. Try reducing demand or allowing more flexibility.")
else:
    
    print(f"Best Cost: ${best_cost}")
    print(f"Full-time tellers used: {best_combo['F']}")
    print("Part-time assignments:")
    for k in range(9, 14):
        print(f"  {k}:00–{k+4}:00 → {best_combo[k]} part-timers")
    # Recompute contributions to show separately
    ft_contrib = get_fulltime_contrib(best_combo['F'])
    pt_contrib = get_parttime_contrib({k: best_combo[k] for k in range(9, 14)})
    total_coverage = combine_contrib(ft_contrib, pt_contrib)

    print("Hourly Coverage Breakdown:")
    print("Hour | Full-time | Part-time | Total")
    print("--------------------------------------")
    for h in range(9, 17):
        f = ft_contrib[h]
        p = pt_contrib[h]
        t = total_coverage[h]
        print(f"{h}:00 |     {f:<9} {p:<10} {t}")

## Example given in the question:
## Please use this for your reference for the first time 
## Enter hourly teller demand from 9 AM to 5 PM (8 hours total):
#  Hour 9:00 - 10:00 → 10
#  Hour 10:00 - 11:00 → 12
#  Hour 11:00 - 12:00 → 14
#  Hour 12:00 - 13:00 → 16
#  Hour 13:00 - 14:00 → 18
#  Hour 14:00 - 15:00 → 17
#  Hour 15:00 - 16:00 → 15
#  Hour 16:00 - 17:00 → 10

Enter hourly teller demand from 9 AM to 5 PM (8 hours total):
Note: demand range b/w 0 to 19


  Hour 9:00 - 10:00 →  10
  Hour 10:00 - 11:00 →  12
  Hour 11:00 - 12:00 →  14
  Hour 12:00 - 13:00 →  16
  Hour 13:00 - 14:00 →  18
  Hour 14:00 - 15:00 →  17
  Hour 15:00 - 16:00 →  15
  Hour 16:00 - 17:00 →  10


Best Cost: $1348
Full-time tellers used: 9
Part-time assignments:
  9:00–13:00 → 1 part-timers
  10:00–14:00 → 2 part-timers
  11:00–15:00 → 2 part-timers
  12:00–16:00 → 7 part-timers
  13:00–17:00 → 2 part-timers
Hourly Coverage Breakdown:
Hour | Full-time | Part-time | Total
--------------------------------------
9:00 |     9         1          10
10:00 |     9         3          12
11:00 |     9         5          14
12:00 |     4         12         16
13:00 |     5         13         18
14:00 |     9         11         20
15:00 |     9         9          18
16:00 |     9         2          11
